In [7]:
import time
import os
import torch
import sys
from options.test_options import TestOptions
from data.test_dataset import testDataSet

sys.argv = ["test.py", "--dataroot" ,"/scratch/user/jiangziyu/test/" ,"--model" ,"test" ,"--dataset_mode" ,"single" ,"--learn_residual" ,"--resize_or_crop" ,"False"]

opt = TestOptions().parse()
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

data_set = testDataSet()
data_set.initialize(opt)
data_loader = torch.utils.data.DataLoader(
            data_set,
            batch_size=opt.batchSize,
            shuffle=not opt.serial_batches,
            num_workers=int(opt.nThreads))

['test.py', '--dataroot', '/scratch/user/jiangziyu/test/', '--model', 'test', '--dataset_mode', 'single', '--learn_residual', '--resize_or_crop', 'False']
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
dataroot: /scratch/user/jiangziyu/test/
dataset_mode: single
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: 256
gan_type: wgan-gp
gpu_ids: [0]
how_many: 5000
input_nc: 3
isTrain: False
learn_residual: True
loadSizeX: 640
loadSizeY: 360
max_dataset_size: inf
model: test
nThreads: 2
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
no_dropout: False
no_flip: False
norm: instance
ntest: inf
output_nc: 3
phase: test
resize_or_crop: False
results_dir: ./results/
serial_batches: False
which_direction: AtoB
which_epoch: latest
which_model_netD: basic
which_model_netG: resnet_9blocks
-------------- End ----------------


In [8]:
from models.test_model_cal import TestModel
from util.visualizer import Visualizer
from pdb import set_trace as st
from util import html
from util.metrics import PSNR
import ssim
from PIL import Image

# test
avgPSNR = 0.0
avgSSIM = 0.0
counter = 0

model = TestModel()
model.initialize(opt)

for i, data in enumerate(dataset):
    if i >= opt.how_many:
        break
    counter = i+1
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    avgPSNR += PSNR(visuals['fake_B'],visuals['real_B'])
    pilFake = Image.fromarray(visuals['fake_B'])
    pilReal = Image.fromarray(visuals['real_B'])
    avgSSIM += SSIM(pilFake).cw_ssim_value(pilReal)
    print('process image... %s' % str(i))
    if(counter % 50 == 0):
        print('PSNR = %f, SSIM = %f' % (avgPSNR/counter, avgSSIM/counter))

avgPSNR /= counter
avgSSIM /= counter
print('PSNR = %f, SSIM = %f' % (avgPSNR, avgSSIM))

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518241554738/work/torch/lib/THC/generic/THCStorage.cu:58